In [14]:
import numpy as np
import pandas as pd
import sklearn.model_selection as skm

# <span style='color:crimson'>1.</span> Load Data

Fonte: https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame

5. Number of Instances: 958 (legal tic-tac-toe endgame boards)

6. Number of Attributes: 9, each corresponding to one tic-tac-toe square

7. Attribute Information: (`x`=player x has taken, `o`=player o has taken, `b`=blank)

    1. top-left-square: {x,o,b}
    2. top-middle-square: {x,o,b}
    3. top-right-square: {x,o,b}
    4. middle-left-square: {x,o,b}
    5. middle-middle-square: {x,o,b}
    6. middle-right-square: {x,o,b}
    7. bottom-left-square: {x,o,b}
    8. bottom-middle-square: {x,o,b}
    9. bottom-right-square: {x,o,b}
    - Class: {`positive`,`negative`}

8. Missing Attribute Values: None

9. Class Distribution: About 65.3% are positive (i.e., wins for "x")

In [2]:
with open('tic-tac-toe_data','rb') as f:
    raw_data = f.readlines()

dataset = np.empty((len(raw_data), 10), dtype=str) # number of samples x (9 positions + result) of type str
for i, sample in enumerate(raw_data):
    sample = sample.decode('utf-8')                # from bytes to str
    dataset[i,:] = sample.strip('\n').split(',')   # add sample

In [3]:
dataset

array([['x', 'x', 'x', ..., 'o', 'o', 'p'],
       ['x', 'x', 'x', ..., 'x', 'o', 'p'],
       ['x', 'x', 'x', ..., 'o', 'x', 'p'],
       ...,
       ['o', 'x', 'o', ..., 'o', 'x', 'n'],
       ['o', 'x', 'o', ..., 'o', 'x', 'n'],
       ['o', 'o', 'x', ..., 'x', 'x', 'n']], dtype='<U1')

# <span style='color:crimson'>1.</span> CART cost function and Gini Index

https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [4]:
# Returns: CART cost function, predicted class
def CART_cost(y, w, idx):
    children = (idx,~idx)                    # left and right child in tree
    ch_C = []                                # predicted class for each child
    ch_P = []                                # probabilities for each child
    ch_G = []                                # Gini index for each child
    w_sum = np.sum(w)                        # sum of all weights
    J = 0                                    # CART cost function
    for child in children:
        # In case child is empty
        if all(child==False):
            ch_C.append(None)
            ch_G.append(None)
            ch_P.append(None)
            continue
            
        w_ch = w[child]                      # weights in node
        y_ch = y[child]                      # classes in node
        w_sum_ch = np.sum(w_ch)              # sum all weights
        G = 1                                # initial Gini index
        max_w_sum_ch_cls = 0                 # maximum weight-sum of child's classes
        for cls in set(y_ch):
            w_sum_ch_cls = np.sum(w_ch[y_ch==cls]) # weight-sum of child's class
            G -= (w_sum_ch_cls/w_sum_ch)**2        # squared of weighted-sum for each class
            if w_sum_ch_cls > max_w_sum_ch_cls:    # weight-sum of child's class greater than current max
                max_w_sum_ch_cls = w_sum_ch_cls
                pred_cls = cls
                
        ch_C.append(pred_cls) # child's predicted class
        ch_G.append(G)        # child's Gini index
        ch_P.append(max_w_sum_ch_cls/w_sum_ch) # child's probability (predicted-class sum of weights/total)
        J += G*w_sum_ch/w_sum # parent node's CART cost
        
    return J, ch_C, ch_G, ch_P

In [5]:
# Classification CART 
def create_node(X, y, w):
    # For each attribute
    J_min = np.inf
    for att in range(X.shape[-1]):
        # For each value of attribute
        for val in set(X[:,att]):
            # Split dataset
            idx = X[:,att]==val # For regression CART, use < instead of ==

            # Evaluate (weighted) CART cost function of each child
            J, children_Preds, children_Ginis, children_Probs = CART_cost(y, w, idx)
            print('X%d == %s Gini=%4.3f'%((att+1), val, J))

            # If it is the best GINI so far, replace best node (which is a lambda function)
            if J < J_min:
                J_min = J
                node_att = att
                node_val = val
                node_ch_Preds = children_Preds
                node_ch_Probs = children_Probs
                node_ch_Ginis = children_Ginis
                node = {'Explanation': 'x[%d] == %s'%(node_att, node_val),
                        'Condition': lambda x: x[node_att]==node_val, 
                        'CART_cost': J_min,
                        'Prediction': {True: node_ch_Preds[0], False: node_ch_Preds[1]},
                        'Children_Probs': {True: node_ch_Probs[0], False: node_ch_Probs[1]},
                        'Children_Ginis': {True: node_ch_Ginis[0], False: node_ch_Ginis[1]}
                       }

    return node

evaluate_node = lambda node,x: node['Prediction'][node['Condition'](x)]

In [6]:
X, y = dataset[:,:-1], dataset[:,-1]
w = np.ones(y.shape) # weights
node = create_node(X, y, w)

X1 == x Gini=0.449
X1 == o Gini=0.444
X1 == b Gini=0.452
X2 == x Gini=0.448
X2 == o Gini=0.451
X2 == b Gini=0.452
X3 == x Gini=0.449
X3 == o Gini=0.444
X3 == b Gini=0.452
X4 == x Gini=0.448
X4 == o Gini=0.451
X4 == b Gini=0.452
X5 == b Gini=0.452
X5 == o Gini=0.401
X5 == x Gini=0.414
X6 == b Gini=0.452
X6 == o Gini=0.451
X6 == x Gini=0.448
X7 == x Gini=0.449
X7 == o Gini=0.444
X7 == b Gini=0.452
X8 == x Gini=0.448
X8 == o Gini=0.451
X8 == b Gini=0.452
X9 == x Gini=0.449
X9 == o Gini=0.444
X9 == b Gini=0.452


In [7]:
[print((key,val)) for key,val in node.items()]

('Explanation', 'x[4] == o')
('Condition', <function create_node.<locals>.<lambda> at 0x7fe9c3d539d8>)
('CART_cost', 0.40054542845980845)
('Prediction', {True: 'n', False: 'p'})
('Children_Probs', {True: 0.5647058823529412, False: 0.7734627831715211})
('Children_Ginis', {True: 0.4916262975778547, False: 0.3504362124401713})


[None, None, None, None, None, None]

In [8]:
assert evaluate_node(node, ['x','x','o','b','o','b','o','x','o']) == 'n' # true negative
assert evaluate_node(node, ['x','x','o','b','o','b','o','o','o']) == 'n' # false negative
assert evaluate_node(node, ['x','x','o','x','b','b','x','b','o']) == 'p' # false positive
assert evaluate_node(node, ['x','x','o','x','x','b','x','o','o']) == 'p' # true positive

In [9]:
hit_rate = lambda y_pred,y_real: float(np.sum(y_pred==y_real)/len(y_pred))

# <span style='color:crimson'>1.</span> Árvore de Decisão

# <span style='color:crimson'>2.</span> *AdaBoost*

# <span style='color:crimson'>3.</span> Dataset

# <span style='color:crimson'>4.</span> Experimentos

- Foram utilizadas funcoes para separar conjuntos de treino/teste e treino/val

In [91]:
n = len(y)                   # numero de amostras
p_prop = sum(y=='p') / n     # proporcao de positivos
n_prop = sum(y=='n') / n     # proporcao de negativos
p_idx = np.where(y=='p')[0]  # indices dos positivos
n_idx = np.where(y=='n')[0]  # indices dos negativos

# Numero de amostras de cada tipo que devem haver nos te
nt = int(round(0.15*n))
npt = int(round(nt * p_prop))
nnt = nt - npt

# 10 testes
splits = []
prev_test_idx = []
for i in range(10):
    np.random.seed(i)
    p_idx_test = np.random.choice(p_idx, npt, replace=False)
    n_idx_test = np.random.choice(n_idx, nnt, replace=False)
    p_idx_train_val = np.setdiff1d(p_idx, p_idx_test)
    n_idx_train_val = np.setdiff1d(n_idx, n_idx_test)
    
    test_idx = np.concatenate((p_idx_test, n_idx_test))
    train_val_idx = np.setdiff1d(np.arange(n), test_idx)
    
    # Validacao: treino/val != teste
    assert len(np.intersect1d(train_val_idx, test_idx))==0
    for pti in prev_test_idx:
        print(len(np.intersect1d(pti, test_idx)), end=' / ')
    prev_test_idx.append(test_idx)
    print()
    
    X_train_val, X_test = X[train_val_idx], X[test_idx]
    y_train_val, y_test = y[train_val_idx], y[test_idx]
    
    # Imprime
    print('Teste #%d' % (i+1))
    print('Train_val["p" "n"]: [%3d %3d] = %3d'%(sum(y_train_val=='p'),sum(y_train_val=='n'),len(y_train_val)))
    print('Test["p" "n"]:      [%3d %3d] = %3d'%(sum(y_test=='p'),     sum(y_test=='n'),     len(y_test)))
    
    np.random.shuffle(p_idx_train_val)
    p_idx_val = np.array_split(p_idx_train_val, 5)
    np.random.shuffle(n_idx_train_val)
    n_idx_val = np.array_split(n_idx_train_val, 5)
    val = [np.concatenate((p_idx_val[k],n_idx_val[k])) for k in range(5)]
    train = [np.setdiff1d(train_val_idx, val[k]) for k in range(5)]
    
    # Salva
    splits.append({'test': test_idx, 'val': val, 'train': train})
    
    prev_val_idx=[]
    for k in range(5):
        train_idx = train[k]
        val_idx = val[k]
    
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Validacao: treino!=val e conjuntos de validacao sao mutuamente exclusivos
        assert len(np.intersect1d(train_idx, val_idx))==0
        assert len(np.intersect1d(train_idx, test_idx))==0
        assert len(np.intersect1d(val_idx, test_idx))==0
        for pvi in prev_val_idx:
            assert len(np.intersect1d(pvi, val_idx))==0
        prev_val_idx.append(val_idx)
            
        # Imprime
        print('\tFold #%d' % k)
        print('\tTrain["p" "n"]: [%3d %3d] = %3d' % (sum(y_train=='p'), sum(y_train=='n'), len(y_train)))
        print('\tVal["p" "n"]:   [%3d %3d] = %3d' % (sum(y_val=='p'),   sum(y_val=='n'),   len(y_val)))


Teste #1
Train_val["p" "n"]: [532 282] = 814
Test["p" "n"]:      [ 94  50] = 144
	Fold #0
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #1
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #2
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #3
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #4
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
25 / 
Teste #2
Train_val["p" "n"]: [532 282] = 814
Test["p" "n"]:      [ 94  50] = 144
	Fold #0
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #1
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #2
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #3
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #4
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
24 / 19 / 
Teste #3
Train_val["p" "n"]: [532 282] = 814
Test["p" "n"]:      [ 94 

In [90]:
len(tests[1]['train'][4])

652

## <span style='color:orange'>1.1.</span> Design dos experimentos

- 10 testes, cada um com um conjunto de teste e uma combinacao de folds 
    - Utilizar np.random.seed(#) para cada teste
    
- Testar distribuicao de classes (alternativa: mudar métrica - e.g., AUC)
    - Completamente aleatorio
    - Stratified shuffle split / stratified k-fold (mantem proporcao de classes)
    - Oversampling/undersampling para tratar class imbalance
    
- Há um máximo de 27 stumps (3 possibilidades para 9 posicoes)
    
<span style='color:red'>[SO COMENTAR] - Vou utilizar stratified</span>

## <span style='color:orange'>1.2.</span> Bias-variance tradeoff

- Grafico #1: Erro de treino/validacao/teste em funcao do numero de preditores - a cada preditor adicionado faz a avaliaca da validacao cruzada e do teste
    - Utiliza 1 (ou uns 3) dos testes (dentre os 10 totais) e salva os erros

## <span style='color:orange'>1.3.</span>  Erro vs. Gini vs. Entropia - slide 127

- Utiliza todos os testes, com o mesmo critério de CV e tira a média

## <span style='color:orange'>1.4.</span> Curvas de aprendizado

- Pega uns 3 testes, e faz:
    - A cada iteracao, acrescenta mais N amostras (manter proporcao aproximada)
    - Faz todo o processo, computando o erro de treinamento/validacao/teste, igual em 1.2

# <span style='color:crimson'>5.</span> Análise de Resultados

# <span style='color:crimson'>6.</span> Conclusão

## <span style='color:orange'>1.1.</span> Subitem